In [1]:
#!/usr/bin/env python
# coding: utf-8
#######################################################
import sys
# !{sys.executable} -m pip install openpyxl
#######################################################
#Import packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns


# Make numpy printouts easier to read.
# np.set_printoptions(precision=3, suppress=True)

import numpy as np
import os
import pickle
import pandas as pd
import sys
from time import time
import matplotlib.pyplot as plt

import shutil
import random
import sklearn
from shutil import copy
import csv
import h5py as h5
import seaborn as sns; sns.set(style="ticks", color_codes=True)
from scipy import stats
#ML Stuff
import joblib

In [2]:
angles=np.arange(106,181,1)
E1Dict=pd.read_csv("E2.csv").rename(columns={'Unnamed: 0':'Label'}).set_index('Label')
'''
First Filter Step: Just the contributers
'''
# with open('type_drop.pickle', 'rb') as handle:
#     type_drop = pickle.load(handle)


with open('keys.pickle', 'rb') as handle:
    keys = pickle.load(handle)

l=keys
l=[i for i in l if float(i)>=110.0 and float(i)<=170.0]
print(len(l))

gen_names=['MP2_pair', 'MP2_Denom', 
           'MP2_amp', 'MP2_amp_mag', 'MP2_amp_sign',
           'From_Same_Orbital', 'To_Same_Orbital', 
           'Jia1', 'Jia2', 'Kia1', 'Kia2', 
           'two_el',  
           'Jia1mag', 'Jia2mag', 'Kia1mag', 'Kia2mag', 
           'Jpp', 'Jqq', 'Jrr', 'Jss', 
           'Kpp', 'Kqq', 'Krr', 'Kss', 
           'h_pp', 'h_qq', 'h_rr', 'h_ss', 
           'h_qp', 'h_sr', 'h_qr', 'h_sp', 
           'MO_Fock_Mat_P_P', 'MO_Fock_Mat_Q_Q', 'MO_Fock_Mat_S_S', 'MO_Fock_Mat_R_R', 
           'MO_Fock_Mat_Q_P', 'MO_Fock_Mat_S_R', 'MO_Fock_Mat_Q_R', 'MO_Fock_Mat_S_P', 
           'MO_Energy_P', 'MO_Energy_Q', 'MO_Energy_R', 'MO_Energy_S', 
           'MO_OCC_P', 'MO_OCC_Q', 'MO_OCC_R', 'MO_OCC_S']

pair_feats=['MP2_pair','From_Same_Orbital',
           'Jqq', 'Jss', 
           'Kqq', 'Kss', 
           'h_qq', 'h_ss', 
           'MO_Fock_Mat_Q_Q', 'MO_Fock_Mat_S_S',
           'MO_Energy_Q', 'MO_Energy_S', 
           'MO_OCC_Q', 'MO_OCC_S']


def gen_pair_Y(typ):
    Y=pd.read_pickle(f'typ{typ}_targets.pickle').astype(float)
# Needs to be qs, we're summing over the occupied orbitals    
    Y_pair_set=list(set(['_'.join((i.split('_')[0],i.split('_')[1]))+'_' for i in Y.index.tolist()]))
    Y_pair_df=pd.concat([Y[Y.index.str.find(j)==0].sum() for j in Y_pair_set],axis=1)
    Y_pair_df.columns=list(set(['_'.join((i.split('_')[0],i.split('_')[1])) for i in Y.index.tolist()]))
    return Y_pair_df.T.sort_index()        



121


In [3]:
min(l),max(l)

('110.0', '170.0')

In [4]:
# np.round((100*(gen_pair_Y('B')/gen_pair_Y('B').sum())).sort_values(by='106.0'),4)

In [5]:
def gen_pair_X(typ):
    Y=pd.read_pickle(f'typ{typ}_targets.pickle').astype(float)
    X=pd.read_pickle(f'typ{typ}_feats.pickle').astype(float)[pair_feats]
    # Needs to be qs, we're summing over the occupied orbitals    
    Y_pair_set=list(set(['_'.join((i.split('_')[0],i.split('_')[1]))+'_' for i in Y.index.tolist()]))

    X_df=pd.DataFrame(columns=pair_feats,index=pd.MultiIndex.from_tuples(sum([[(i,'_'.join(j.split('_')[0:2])) for j in Y_pair_set] for i in l],[]))).sort_index()
    for i in l:
        for j in Y_pair_set:
            X_df.loc[(i,'_'.join(j.split('_')[0:2])),'MP2_pair']=X.loc[i][X.loc[i].index.str.find(j)==0]['MP2_pair'].sum()
            X_df.loc[(i,'_'.join(j.split('_')[0:2])),['From_Same_Orbital','Jqq', 'Jss', 'Kqq', 'Kss', 'h_qq', 'h_ss', 'MO_Fock_Mat_Q_Q', 'MO_Fock_Mat_S_S','MO_Energy_Q', 'MO_Energy_S', 'MO_OCC_Q', 'MO_OCC_S']]= X.loc[i][X.loc[i].index.str.find(j)==0][['From_Same_Orbital','Jqq', 'Jss', 'Kqq', 'Kss', 'h_qq', 'h_ss', 'MO_Fock_Mat_Q_Q', 'MO_Fock_Mat_S_S','MO_Energy_Q', 'MO_Energy_S', 'MO_OCC_Q', 'MO_OCC_S']].iloc[0]
    return X_df.astype(float)

In [6]:
# Y=pd.concat([gen_pair_Y(typ) for typ in type_drop])


# Y=pd.read_pickle('targets.pickle')
Y=pd.read_pickle('fixed_targets.pickle')

In [7]:
# X=pd.concat([gen_pair_X(typ) for typ in type_drop])
# X=pd.read_pickle('feats.pickle')
X=pd.read_pickle('fixed_feats.pickle').drop(columns=['pair_energy'])

In [8]:
X.columns

Index(['From_Same_Orbital', 'coulomb', 'screen1_1', 'screen1_2', 'screen1_3',
       'screen1_4', 'screen2_1', 'screen2_2', 'screen2_3', 'screen2_4',
       'eijab_1', 'eijab_2', 'eijab_3', 'eijab_4', 'screenvirt_1',
       'screenvirt_2', 'screenvirt_3', 'screenvirt_4', 'Fr1', 'Fr2', 'Fr3',
       'Fr4', 'Fs1', 'Fs2', 'Fs3', 'Fs4', 'occr1', 'occr2', 'occr3', 'occr4',
       'occs1', 'occs2', 'occs3', 'occs4', 'SCFFr1', 'SCFFr2', 'SCFFr3',
       'SCFFr4', 'SCFFs1', 'SCFFs2', 'SCFFs3', 'SCFFs4', 'SCFOCCr1',
       'SCFOCCr2', 'SCFOCCr3', 'SCFOCCr4', 'SCFOCCs1', 'SCFOCCs2', 'SCFOCCs3',
       'SCFOCCs4', 'hrr1', 'hrr2', 'hrr3', 'hrr4', 'hss1', 'hss2', 'hss3',
       'hss4', 'hpp', 'hqq', 'Fp', 'Fq', 'occp', 'occq', 'SCFFp', 'SCFFq',
       'SCFOCCp', 'SCFOCCq'],
      dtype='object')

In [131]:
#ML Stuff
from sklearn import metrics, svm
# from sklearn import preprocessing
# Feature selection
from sklearn import feature_selection
# Preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
# Metrics
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn import preprocessing as skpreprocessing
from sklearn.model_selection import GridSearchCV

In [132]:
with open('test_ind.pickle', 'rb') as handle:
    test_ind = pickle.load(handle)

with open('train_ind.pickle', 'rb') as handle:
    train_ind = pickle.load(handle)



# train_ind,test_ind=train_test_split(l, test_size=0.3, random_state=0)
num_train=[int(i) for i in np.linspace(10,len(train_ind),19)]
y_train=Y[train_ind]
#     drop=y_train.loc[y_train.T.describe().loc['mean'].abs().sort_values().apply(lambda x: abs(np.log10(x)))<20].index.tolist()
drop_idx=min(y_train.columns)
drop=(np.round(100*(y_train[drop_idx].sort_values()/y_train[drop_idx].sum()),6)!=0).index

In [133]:
num_train

[10, 14, 18, 22, 26, 30, 34, 38, 42, 47, 51, 55, 59, 63, 67, 71, 75, 79, 84]

In [134]:
# plt.scatter(np.array(sorted(test_ind)).astype(float),np.zeros(len(test_ind)))
# plt.scatter(np.array(sorted(train_ind)).astype(float),np.zeros(len(train_ind)))
# plt.show()

In [138]:
# find nan
# X[pd.isnull(X).any(1)]

In [139]:
X_train=X.loc[train_ind]
X_test=X.loc[test_ind]

y_train=Y[train_ind].T.stack()
y_test=Y[test_ind].T.stack()



In [140]:
len(test_ind),len(train_ind)

(37, 84)

In [141]:
X_train.shape,y_train.shape

((7896, 68), (7896,))

In [142]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import TransformedTargetRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
scaler = skpreprocessing.MinMaxScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)



In [73]:
# !{sys.executable} -m pip uninstall tensorflow keras --yes
# !{sys.executable} -m pip install tensorflow keras

In [74]:

# # parameters = {'max_depth':[5,1000,None],
# #               'max_features':['auto','sqrt'],
# #               'bootstrap':[True],
# #               'oob_score':[True],
# #               'n_estimators':[100,700,1000]
# #              }
# # GridSearch=GridSearchCV(RandomForestRegressor(random_state =69), parameters,verbose=1,n_jobs=-1).fit(X_train_scaled,y_train)

# from tensorflow.python import keras
# from keras.models import Sequential
# from keras.layers import Dense, Flatten, Conv2D, Dropout
# from keras.layers import LeakyReLU
# import tensorflow as tf
# from keras import backend as K
# from tensorflow.keras.optimizers import Adam
# #from keras.layers.advanced_activations import LeakyReLU
# from tensorflow.keras import layers
# x=0.0
# # 32 original
# layer_shape=X_train_scaled.shape[1]
# model = Sequential()
# model.add(Dense(128, input_shape=(X_train_scaled.shape[1],)))
# #added 1/10
# model.add(LeakyReLU(alpha=0.05))
# #model.add(Dropout(x))
# model.add(Dense(layer_shape))
# model.add(LeakyReLU(alpha=0.05))
# #model.add(Dropout(x))
# model.add(Dense(layer_shape))
# model.add(LeakyReLU(alpha=0.05))
# model.add(Dropout(x))
# model.add(Dense(layer_shape))
# model.add(LeakyReLU(alpha=0.05))
# model.add(Dropout(x))
# model.add(Dense(layer_shape))
# model.add(LeakyReLU(alpha=0.05))
# model.add(Dropout(x))
# model.add(Dense(layer_shape))
# model.add(LeakyReLU(alpha=0.05))
# model.add(Dense(layer_shape))
# model.add(LeakyReLU(alpha=0.05))
# model.add(Dropout(x))
# model.add(Dense(layer_shape))
# model.add(LeakyReLU(alpha=0.05))
# model.add(Dropout(x))
# model.add(Dense(layer_shape))
# model.add(LeakyReLU(alpha=0.05))
# model.add(Dropout(x))
# model.add(Dense(layer_shape))
# model.add(LeakyReLU(alpha=0.05))
# model.add(Dropout(x))
# model.add(Dense(layer_shape))
# model.add(LeakyReLU(alpha=0.05))
# model.add(Dropout(x))
# model.add(Dense(layer_shape))
# model.add(LeakyReLU(alpha=0.05))
# model.add(Dense(1))
# #Added 1/10

# opt=Adam(learning_rate=1e-5)

# model.compile(optimizer=opt, loss='mean_squared_error',metrics=['mse', 'mae', 'mape'])#,loss='mean_squared_error',)



# # history=model.fit(X_train_scaled, y_train,batch_size=128,epochs=12000)
# history=model.fit(X_train_scaled, y_train,batch_size=128,epochs=100,workers=4)


# import matplotlib.pyplot as plt
# def plot_history(history, label):
#     plt.plot(history.history['loss'])
# #     plt.plot(history.history['val_loss'])
#     plt.title('Loss for %s' % label)
#     plt.ylabel('Loss')
#     plt.xlabel('Epoch')
# #     plt.ylim(0, 1e-8)
#     _= plt.legend(['Train'], loc='upper left')
#     plt.show()
    
# def plot_save(history, label):
#     plt.plot(history.history['loss'])
#     plt.plot(history.history['val_loss'])
#     plt.title('Loss for %s' % label)
#     plt.ylabel('Loss')
#     plt.xlabel('Epoch')
#     plt.ylim(0, 250000)
#     _= plt.legend(['Train','Validation'], loc='upper left')
#     plt.savefig(str(label)+'.png', dpi=200)
    
# plot_history(history, 'Training')
# #plot_save(history, 'Waters 6 Layers 10 Dropout (1layer)')

# print(model.summary())

In [75]:
# y_test_pred=model.predict(X_test_scaled)
# y_train_pred=model.predict(X_train_scaled)
# plt.scatter(y_test_pred,y_test)
# plt.plot(y_test,y_test)
# plt.show()
# plt.scatter(y_train_pred,y_train)
# plt.plot(y_train,y_train)
# plt.show()

In [76]:
# r2_score(y_test_pred,y_test),r2_score(y_train_pred,y_train)


In [77]:
# pred_train_E2=np.sum(y_train_pred.reshape(len(train_ind),-1),axis=1)
# train_E2=np.sum(y_train.values.reshape(len(train_ind),-1),axis=1)
# pred_test_E2=np.sum(y_test_pred.reshape(len(test_ind),-1),axis=1)
# test_E2=np.sum(y_test.values.reshape(len(test_ind),-1),axis=1)
# abs_test=pd.DataFrame(pred_test_E2-test_E2,index=test_ind).abs().sort_index()*627.5
# abs_train=pd.DataFrame(pred_train_E2-train_E2,index=train_ind).abs().sort_index()*627.5

In [78]:
# caspt2=pd.read_csv('caspt2.csv',index_col='Label').drop(columns='Unnamed: 0')
# df_PredE2=pd.DataFrame(pred_test_E2,index=map(float,test_ind))
# df_PredE2_train=pd.DataFrame(pred_train_E2,index=map(float,train_ind))
# casscf=pd.read_csv('casscf.csv',index_col='Label').drop(columns='Unnamed: 0').loc[map(float,test_ind)].rename(columns={'SCF':0})
# plt.plot(caspt2-caspt2.max().values,label='CASPT2')
# plt.scatter(list(map(float,test_ind)),casscf.add(df_PredE2,fill_value=0)-caspt2.max().values,label='Test DDCASPT2',color='red')
# # plt.scatter(list(map(float,train_ind)),pd.read_csv('casscf.csv',index_col='Label').drop(columns='Unnamed: 0').loc[map(float,train_ind)].rename(columns={'SCF':0}).add(df_PredE2_train,fill_value=0)-caspt2.max().values,label='Train DDCASPT2',color='green')
# plt.legend()
# plt.title(r'O$_{3}$ Potential Energy Curve',fontsize=14)
# plt.ylabel('Energy (E$_{h}$)',fontsize=14)
# plt.xlabel('Bond Degrees ($^{\circ}$)',fontsize=14)
# plt.tight_layout()
# # plt.savefig(f'PEC_4_3_1D_VDZP_LC.png',dpi=300)
# plt.show()

In [143]:
from sklearn.kernel_ridge import KernelRidge
parameters = {
    'alpha':np.logspace(-3,1,5),
    'kernel':['rbf','laplacian'],
    'gamma':np.logspace(-3,1,5)
}


GridSearch = GridSearchCV(KernelRidge(),param_grid=parameters,n_jobs=-1,cv=5,verbose=1,scoring='explained_variance').fit(X_train_scaled,y_train)
regr=GridSearch.best_estimator_
# regr=KernelRidge(alpha=0.001, gamma=1.0, kernel='laplacian').fit(X_train_scaled,y_train)
# regr=KNeighborsRegressor(n_neighbors=int(np.log(len(y_train[train_ind[0]]))),leaf_size=1,n_jobs=-1,p=1).fit(X_train_scaled,y_train)
# regr = KNeighborsRegressor(n_neighbors=1,leaf_size=30,n_jobs=-1,p=1).fit(X_train_scaled, y_train)
# regr = TransformedTargetRegressor(regressor=KNeighborsRegressor(n_neighbors=2,leaf_size=1,n_jobs=-1,p=1),transformer=MinMaxScaler())
# regr.fit(X_train_scaled, y_train)
# regr = RandomForestRegressor(max_depth=1000,max_features='sqrt',bootstrap=True,oob_score=True,n_estimators=700,random_state =69,n_jobs=-1).fit(X_train_scaled, y_train)
# regr = RandomForestRegressor(max_features='sqrt',bootstrap=True,oob_score=True,n_estimators=100,random_state =69,n_jobs=-1).fit(X_train_scaled, y_train)
y_train_pred=regr.predict(X_train_scaled)
y_test_pred=regr.predict(X_test_scaled)


pred_train_E2=np.sum(y_train_pred.reshape(len(train_ind),-1),axis=1)
train_E2=np.sum(y_train.values.reshape(len(train_ind),-1),axis=1)
pred_test_E2=np.sum(y_test_pred.reshape(len(test_ind),-1),axis=1)
test_E2=np.sum(y_test.values.reshape(len(test_ind),-1),axis=1)
abs_test=pd.DataFrame(pred_test_E2-test_E2,index=test_ind).abs().sort_index()*627.5
abs_train=pd.DataFrame(pred_train_E2-train_E2,index=train_ind).abs().sort_index()*627.5
(abs_test).plot()
(abs_train).plot()

Fitting 5 folds for each of 50 candidates, totalling 250 fits


<AxesSubplot:>

In [144]:
regr

KernelRidge(alpha=0.001, gamma=10.0, kernel='rbf')

In [145]:
regr.score(X_train_scaled, y_train)

0.9999969869812124

In [146]:
regr.score(X_test_scaled, y_test)

0.9999948594846231

In [147]:
plt.figure(figsize=(10,5))
plt.subplot(1, 2, 1)
fill=np.linspace(min(y_test)-1e-2,max(y_test)+1e-2,20)
plt.plot(fill,fill,color='black',alpha=0.75)
plt.fill_between(fill,fill-0.0016,fill+0.0016,color='gray', alpha=0.3)
plt.fill_between(fill,fill-0.003665,fill+0.003665,color='darkgray', alpha=0.3)

plt.scatter(y_train,y_train_pred,alpha=0.5,marker='.')
plt.xticks(np.round(np.linspace(min(y_train), max(y_train), 4),4),rotation=0)
plt.text(min(y_train), max(y_train)-5e-3, f'R$^{2}$: {r2_score(y_train,y_train_pred):0.3f}',fontsize=12,bbox=dict(facecolor='white', alpha=1.0))
plt.xlim(min(y_train)-1e-3, max(y_train)+1e-3)
plt.ylim(min(y_train)-1e-3, max(y_train)+1e-3)
plt.xlabel('True pair-energies (E$_{h}$)',fontsize=12)
plt.ylabel('Predicted pair-energies (E$_{h}$)',fontsize=12)
plt.title('O$_{3}$ (4,3) VDZP 1D Train Set',fontsize=12)
plt.subplot(1, 2, 2)
fill=np.linspace(min(y_test)-1e-2,max(y_test)+1e-2,20)
plt.plot(fill,fill,color='black',alpha=0.75)
plt.fill_between(fill,fill-0.0016,fill+0.0016,color='gray', alpha=0.3)
plt.fill_between(fill,fill-0.003665,fill+0.003665,color='darkgray', alpha=0.3)
plt.scatter(y_test,y_test_pred,alpha=0.5,marker='.',color='orange')
plt.xticks(np.round(np.linspace(min(y_test), max(y_test), 4),4),rotation=0)
plt.text(min(y_test), max(y_test)-5e-3, f'R$^{2}$: {r2_score(y_test,y_test_pred):0.3f}',fontsize=12,bbox=dict(facecolor='white', alpha=1.0))
plt.xlim(min(y_test)-1e-3, max(y_test)+1e-3)
plt.ylim(min(y_test)-1e-3, max(y_test)+1e-3)
plt.xlabel('True pair-energies (E$_{h}$)',fontsize=12)
plt.ylabel('Predicted pair-energies (E$_{h}$)',fontsize=12)
plt.title('O$_{3}$ (4,3) VDZP 1D Test Set',fontsize=12)

plt.tight_layout()
plt.savefig(f'scaled_new_4_3_1D_VDZP_pair.png',dpi=300)
plt.savefig(f'low_res_scaled_new_4_3_1D_VDZP_pair.png')
plt.show()

In [148]:
plt.scatter(y_test,y_test_pred)
plt.plot(y_test,y_test)
plt.show()
plt.scatter(y_train,y_train_pred)
plt.plot(y_train,y_train)
plt.show()

In [149]:
test_E2

array([-0.10003951, -0.10035274, -0.10025354, -0.09573178, -0.10003053,
       -0.09732632, -0.09562057, -0.0959608 , -0.09977433, -0.10079576,
       -0.09854612, -0.1002094 , -0.09504918, -0.10092113, -0.09684428,
       -0.10134766, -0.09822369, -0.09957182, -0.09798022, -0.10132805,
       -0.10107713, -0.10086676, -0.09665053, -0.09755992, -0.09811517,
       -0.09630118, -0.10051768, -0.10044281, -0.09778553, -0.10126013,
       -0.10029335, -0.09913129, -0.09857403, -0.10096922, -0.0951606 ,
       -0.10013605, -0.09550439])

In [150]:
plt.scatter(E1Dict.loc[np.array(test_ind).astype(float)].values.reshape(-1),test_E2)
plt.scatter(E1Dict.loc[np.array(train_ind).astype(float)].values.reshape(-1),train_E2)

In [151]:
abs_test.describe()

,0
count,37.000000
mean,0.026467
std,0.032930
min,0.000524
25%,0.007541
50%,0.017222
75%,0.033957
max,0.161576


In [152]:
plt.scatter(test_ind,pred_test_E2-test_E2)
plt.plot(test_ind,np.zeros(len(test_ind)))
# plt.ylim(min(pred_test_E2-test_E2),max(pred_test_E2-test_E2))
plt.xticks(test_ind[::5])
plt.show()

In [153]:
df_test=pd.DataFrame({'test_E2':test_E2,'pred_test_E2':pred_test_E2},index=test_ind).sort_index()
df_train=pd.DataFrame({'train_E2':train_E2,'pred_train_E2':pred_train_E2},index=train_ind).sort_index()

In [154]:
# plt.scatter(E1Dict.loc[np.array(test_ind).astype(float)],pred_test_E2)
plt.scatter(test_E2,pred_test_E2)
plt.plot(test_E2,test_E2)

In [155]:
r2_score(E1Dict.loc[np.array(train_ind).astype(float)],pred_train_E2)

0.9986948782169855

In [156]:
r2_score(E1Dict.loc[np.array(test_ind).astype(float)],pred_test_E2)

0.9988854337563

In [157]:
print('Test E2 MAE: %.4E' % mean_absolute_error(test_E2,pred_test_E2), 'Eh')
print('Test E2 RMSE: %.4E' % np.sqrt(mean_squared_error(test_E2,pred_test_E2)), 'Eh','\n')
print('Test E2 MAE: %.4E' % ((mean_absolute_error(test_E2,pred_test_E2))*627.5), 'kcal/mol')
print('Test E2 RMSE: %.4E' % (np.sqrt(mean_squared_error(test_E2,pred_test_E2))*627.5), 'kcal/mol','\n')

print('Train E2 MAE: %.4E' % mean_absolute_error(train_E2,pred_train_E2), 'Eh')
print('Train E2 RMSE: %.4E' % np.sqrt(mean_squared_error(train_E2,pred_train_E2)), 'Eh','\n')
print('Train E2 MAE: %.4E' % ((mean_absolute_error(train_E2,pred_train_E2))*627.5), 'kcal/mol')
print('Train E2 RMSE: %.4E' % (np.sqrt(mean_squared_error(train_E2,pred_train_E2))*627.5), 'kcal/mol','\n')



Test E2 MAE: 4.2178E-05 Eh
Test E2 RMSE: 6.6772E-05 Eh 

Test E2 MAE: 2.6467E-02 kcal/mol
Test E2 RMSE: 4.1899E-02 kcal/mol 

Train E2 MAE: 4.7400E-05 Eh
Train E2 RMSE: 6.6950E-05 Eh 

Train E2 MAE: 2.9744E-02 kcal/mol
Train E2 RMSE: 4.2011E-02 kcal/mol 



In [158]:
plt.scatter(np.array(df_train.sort_index().index).astype(float),df_train.sort_index()['train_E2']-df_train.sort_index()['pred_train_E2'])
plt.scatter(np.array(df_test.sort_index().index).astype(float),df_test.sort_index()['test_E2']-df_test.sort_index()['pred_test_E2'])
plt.hlines(0,min(np.array(l).astype(float)),max(np.array(l).astype(float)))
# plt.xticks(np.arange(0, len(l), step=20))
plt.show()
plt.scatter(df_test.sort_index().index,df_test.sort_index()['test_E2']-df_test.sort_index()['pred_test_E2'])
plt.hlines(0,0,len(test_ind))
plt.xticks(np.arange(0, len(test_ind), step=5))
plt.show()
plt.scatter(np.array(test_ind).astype(float),pd.DataFrame({'test_E2':test_E2,'pred_test_E2':pred_test_E2},index=test_ind)['test_E2'],label='Test')
plt.scatter(np.array(test_ind).astype(float),pd.DataFrame({'test_E2':test_E2,'pred_test_E2':pred_test_E2},index=test_ind)['pred_test_E2'],label='predicted')
plt.plot(E1Dict.index,E1Dict.values,color='r')
plt.legend()
plt.show()

In [159]:
r2_score(train_E2,pred_train_E2),r2_score(test_E2,pred_test_E2)

(0.9986872819377832, 0.9988945035826029)

In [160]:
plt.scatter(test_E2,pred_test_E2,color='orange')
plt.plot(test_E2,test_E2)
plt.plot(E1Dict.values,E1Dict.values)
plt.xlabel('True')
plt.ylabel('Predicted')
plt.show()

In [161]:
# Energies=pd.read_excel(f'scaled_50cent_4_3_VDZP_Test_Energies.xlsx',engine='openpyxl')
# curve=pd.read_excel(f'scaled_50cent_4_3_VDZP_Learning_Curve_1D.xlsx',engine='openpyxl')
test_avg=np.average(test_E2)*627.5
plttest=test_E2*627.5
pltpred=pred_test_E2*627.5
# plt.figure(figsize=(6,6))
plt.scatter(plttest,pltpred,label='Predicted')
fill=np.linspace(min(plttest)-5,max(plttest)+5,20)
plt.plot(fill,fill,color='black',alpha=0.75)
plt.fill_between(fill,fill-1,fill+1,color='gray', alpha=0.3)
plt.fill_between(fill,fill-2.3,fill+2.3,color='darkgray', alpha=0.3)

# plt.text(min(plttest)-9e-1, max(plttest)+1e-1, f"Average E2 captured: {(100-abs((Energies['True']-Energies['PredTestE2'])/Energies['True'])*100).mean():.2f} percent",fontsize='medium',bbox=dict(facecolor='white', alpha=0))
plt.text(min(plttest)-3.5, max(plttest)+3, 'Average Test E$_{2}$:'+f' {test_avg:.2f} kcal/mol',fontsize='medium',bbox=dict(facecolor='white', alpha=0))
plt.text(min(plttest)-3.5, max(plttest)+2, 'Test E$_{2}$ MAE:'+f' {mean_absolute_error(plttest,pltpred):.2f} kcal/mol',fontsize='medium',bbox=dict(facecolor='white', alpha=0))
plt.text(min(plttest)-3.5, max(plttest)+1, 'Test E$_{2}$ RMSE: '+f'{np.sqrt(mean_squared_error(plttest,pltpred)):.2f} kcal/mol',fontsize='medium',bbox=dict(facecolor='white', alpha=0))
plt.text(min(plttest)-3.5, max(plttest)-0.3, 'Test R$^{2}$: '+f'{r2_score(plttest,pltpred):.2f}',fontsize='medium',bbox=dict(facecolor='white', alpha=0))
plt.xlim(min(plttest)-4,max(plttest)+4)
plt.ylim(min(plttest)-4,max(plttest)+4)
plt.title(r'O$_{3}$ 1D (4,3) VDZP',fontsize=14)
plt.xlabel(r'True E$_{2}$ (kcal/mol)',fontsize=14)
plt.ylabel(r'Predicted E$_{2}$ (kcal/mol)',fontsize=14)
plt.tight_layout()
plt.savefig(f'50cent_4_3_1D_VDZP_LC.png',dpi=300)
plt.show()

In [162]:
caspt2=pd.read_csv('caspt2.csv',index_col='Label').drop(columns='Unnamed: 0')
df_PredE2=pd.DataFrame(pred_test_E2,index=map(float,test_ind))
df_PredE2_train=pd.DataFrame(pred_train_E2,index=map(float,train_ind))
casscf=pd.read_csv('casscf.csv',index_col='Label').drop(columns='Unnamed: 0').loc[map(float,test_ind)].rename(columns={'SCF':0})
plt.plot(caspt2-caspt2.max().values,label='CASPT2')
plt.scatter(list(map(float,test_ind)),casscf.add(df_PredE2,fill_value=0)-caspt2.max().values,label='Test DDCASPT2',color='red')
# plt.scatter(list(map(float,train_ind)),pd.read_csv('casscf.csv',index_col='Label').drop(columns='Unnamed: 0').loc[map(float,train_ind)].rename(columns={'SCF':0}).add(df_PredE2_train,fill_value=0)-caspt2.max().values,label='Train DDCASPT2',color='green')
plt.legend()
plt.title(r'O$_{3}$ Potential Energy Curve',fontsize=14)
plt.ylabel('Energy (E$_{h}$)',fontsize=14)
plt.xlabel('Bond Degrees ($^{\circ}$)',fontsize=14)
plt.tight_layout()
plt.savefig(f'PEC_4_3_1D_VDZP_LC.png',dpi=300)
plt.show()

In [163]:
sorted_seventy=pd.DataFrame({'Test':plttest,'Pred':pltpred,'Diff':plttest-pltpred},index=test_ind).abs().sort_values(by='Diff',ascending=False)

In [164]:
sorted_seventy

,Test,Pred,Diff
170.0,60.769783,60.608207,0.161576
115.0,60.215400,60.332057,0.116657
157.5,62.909099,62.818991,0.090108
164.0,61.855206,61.909222,0.054016
148.0,63.583349,63.624916,0.041568
162.0,62.204887,62.244512,0.039625
114.0,60.071689,60.033328,0.038361
123.0,61.360419,61.322919,0.037499
158.5,62.769159,62.734049,0.035110
154.5,63.249340,63.215383,0.033957


In [165]:
r2_score(sorted_seventy['Test'].iloc[1:],sorted_seventy['Pred'].iloc[1:])

0.9993205996830744

In [166]:
# Check
plt.scatter(E1Dict,Y.sum())

In [167]:
# Create the dataset
rng = np.random.default_rng(42)
# factor_dict=pd.read_excel('factor_dict.xlsx')
R2=[]
MAE=[]
RMSE=[]
kcalMAE=[]
kcalRMSE=[]
Train_R2=[]
Train_MAE=[]
Train_RMSE=[]
Train_kcalMAE=[]
Train_kcalRMSE=[]
Num_Test=[]
Num_Train=[]
for k in num_train:
    TestE2_list=[]
    PredE2_list=[]
    Train_E2_list=[]
    Train_PredE2_list=[]
    testsubs_list=[]
    trainsubs_list=[]


    print(f'Number of Train {k}')
#         trainsubs=random.sample(train_ind, k)
    trainsubs=train_ind[0:k]
    testsubs=test_ind
    Num_Test.append(len(testsubs))
    Num_Train.append(k)
    print(f'Number of test {len(testsubs)}')
#         for typ in ['A','B','C','D','E','F','G','H']:
    
    y=Y
    X_test_stacked=X.loc[testsubs]
    X_train_stacked=X.loc[trainsubs]
    y_test_stacked=y[testsubs].T.stack()
    y_train_stacked=y[trainsubs].T.stack()


    test_labels=testsubs
    train_labels=trainsubs
    scaler = MinMaxScaler().fit(X_train_stacked)
    X_train_scaled = scaler.transform(X_train_stacked)
    X_test_scaled = scaler.transform(X_test_stacked)
    print(X_train_scaled.shape)
#     reg8=KNeighborsRegressor(n_neighbors=int(np.log(len(y[trainsubs[0]]))),leaf_size=1,n_jobs=-1,p=1).fit(X_train_scaled,y_train_stacked)    
    reg8=regr.fit(X_train_scaled,y_train_stacked)
#     reg8=RandomForestRegressor(max_features='sqrt',bootstrap=True,oob_score=True,n_estimators=700,random_state =69,n_jobs=-1).fit(X_train_scaled, y_train_stacked)
    
    print(reg8.get_params)
    y_test_trimpred=reg8.predict(X_test_scaled)
    y_train_trimpred=reg8.predict(X_train_scaled)


    print(f'pair_e2 test R2: %.4f' %r2_score(y_test_stacked,y_test_trimpred))
    print(f'pair_e2 train R2: %.4f' %r2_score(y_train_stacked,y_train_trimpred))  
    PredE2=np.sum(y_test_trimpred.reshape(len(testsubs),-1),axis=1)
    TestE2=np.sum(y_test_stacked.values.reshape(len(testsubs),-1),axis=1)
    Train_PredE2=np.sum(y_train_trimpred.reshape(len(trainsubs),-1),axis=1)
    Train_E2=np.sum(y_train_stacked.values.reshape(len(trainsubs),-1),axis=1)
    print(f'E2 test R2: %.4f' %r2_score(TestE2,PredE2))
    print(f'E2 train R2: %.4f' %r2_score(Train_E2,Train_PredE2))     
    TestE2_list.append(TestE2)
    PredE2_list.append(PredE2)
    Train_E2_list.append(Train_E2)
    Train_PredE2_list.append(Train_PredE2)
    testsubs_list.append(testsubs)
    trainsubs_list.append(trainsubs)
    

    R2.append(r2_score(TestE2,PredE2))
    MAE.append(mean_absolute_error(TestE2,PredE2))
    RMSE.append(np.sqrt(mean_squared_error(TestE2,PredE2)))
    kcalMAE.append(627.5*mean_absolute_error(TestE2,PredE2))
    kcalRMSE.append(627.5*np.sqrt(mean_squared_error(TestE2,PredE2)))

    Train_R2.append(r2_score(Train_E2,Train_PredE2))
    Train_MAE.append(mean_absolute_error(Train_E2,Train_PredE2))
    Train_RMSE.append(np.sqrt(mean_squared_error(Train_E2,Train_PredE2)))
    Train_kcalMAE.append(627.5*mean_absolute_error(Train_E2,Train_PredE2))
    Train_kcalRMSE.append(627.5*np.sqrt(mean_squared_error(Train_E2,Train_PredE2)))

    print('Test E2 MAE: %.4E' % mean_absolute_error(TestE2,PredE2), 'Eh')
    print('Test E2 RMSE: %.4E' % np.sqrt(mean_squared_error(TestE2,PredE2)), 'Eh','\n')
    print('Test E2 MAE: %.4E' % ((mean_absolute_error(TestE2,PredE2))*627.5), 'kcal/mol')
    print('Test E2 RMSE: %.4E' % (np.sqrt(mean_squared_error(TestE2,PredE2))*627.5), 'kcal/mol','\n')

    print('Train E2 MAE: %.4E' % mean_absolute_error(Train_E2,Train_PredE2), 'Eh')
    print('Train E2 RMSE: %.4E' % np.sqrt(mean_squared_error(Train_E2,Train_PredE2)), 'Eh','\n')
    print('Train E2 MAE: %.4E' % ((mean_absolute_error(Train_E2,Train_PredE2))*627.5), 'kcal/mol')
    print('Train E2 RMSE: %.4E' % (np.sqrt(mean_squared_error(Train_E2,Train_PredE2))*627.5), 'kcal/mol','\n')
    
    pd.DataFrame({'TestE2':TestE2,'testsubs':testsubs,'PredE2':PredE2}).to_excel(f'big_scaled_super_4_3_VDZP_Test_Energies.xlsx')
    pd.DataFrame({'Num_Test':Num_Test,'Num_Train':Num_Train,'Test_R2':R2,'Test_MAE (kcal/mol)':kcalMAE,'Test_RMSE (kcal/mol)':kcalRMSE,'Test_MAE (Eh)':MAE,'Test_RMSE (Eh)':RMSE,'Train_R2':Train_R2,'Train_MAE':Train_MAE,'Train_RMSE':Train_RMSE,'Train_kcalMAE':Train_kcalMAE,'Train_kcalRMSE':Train_kcalRMSE}).sort_values('Num_Train').to_excel(f'big_scaled_super_4_3_VDZP_Learning_Curve_1D.xlsx')





Number of Train 10
Number of test 37
(940, 68)
<bound method BaseEstimator.get_params of KernelRidge(alpha=0.001, gamma=10.0, kernel='rbf')>
pair_e2 test R2: 0.9982
pair_e2 train R2: 1.0000
E2 test R2: -2.0978
E2 train R2: 0.9907
Test E2 MAE: 1.8458E-03 Eh
Test E2 RMSE: 3.5346E-03 Eh 

Test E2 MAE: 1.1582E+00 kcal/mol
Test E2 RMSE: 2.2180E+00 kcal/mol 

Train E2 MAE: 1.1151E-04 Eh
Train E2 RMSE: 1.3767E-04 Eh 

Train E2 MAE: 6.9972E-02 kcal/mol
Train E2 RMSE: 8.6390E-02 kcal/mol 

Number of Train 14
Number of test 37
(1316, 68)
<bound method BaseEstimator.get_params of KernelRidge(alpha=0.001, gamma=10.0, kernel='rbf')>
pair_e2 test R2: 0.9996
pair_e2 train R2: 1.0000
E2 test R2: 0.5841
E2 train R2: 0.9908
Test E2 MAE: 6.8428E-04 Eh
Test E2 RMSE: 1.2951E-03 Eh 

Test E2 MAE: 4.2939E-01 kcal/mol
Test E2 RMSE: 8.1265E-01 kcal/mol 

Train E2 MAE: 1.2338E-04 Eh
Train E2 RMSE: 1.5964E-04 Eh 

Train E2 MAE: 7.7419E-02 kcal/mol
Train E2 RMSE: 1.0018E-01 kcal/mol 

Number of Train 18
Number of

In [168]:
from mpl_toolkits.axes_grid.inset_locator import (inset_axes, InsetPosition,mark_inset)
      
df = pd.read_excel(f'big_scaled_super_4_3_VDZP_Learning_Curve_1D.xlsx',engine='openpyxl')
test=pd.read_excel('big_scaled_super_4_3_VDZP_Test_Energies.xlsx',engine='openpyxl').set_index('testsubs').drop(columns='Unnamed: 0')
VDZP_perc_train=[float(f'{i*100:0.0f}') for i in list(df['Num_Train']/df['Num_Train'].iloc[-1])]
VDZP_test_R2=df['Test_R2']
VDZP_Train_R2=df['Train_R2']
VDZP_Test_MAE=df['Test_MAE (kcal/mol)']
VDZP_Test_RMSE=df['Test_RMSE (kcal/mol)']
VDZP_train_MAE=df['Train_kcalMAE']
VDZP_train_RMSE=df['Train_kcalRMSE']
# plt.plot(VDZP_perc_train,VDZP_Test_MAE,color='b',label=f'Test 1D VDZP MAE',marker='x',ls='-.')
# plt.plot(VDZP_perc_train,VDZP_Test_RMSE,color='b',label=f'Test 1D VDZP RMSE',marker='+',ls='-.')
# plt.plot(VDZP_perc_train,VDZP_train_MAE,color='orange',label=f'Train 1D VDZP MAE',marker='x',ls='-.')
# plt.plot(VDZP_perc_train,VDZP_train_RMSE,color='orange',label=f'Train 1D VDZP RMSE',marker='+',ls='-.')
plt.plot(VDZP_perc_train,VDZP_Test_MAE,color='b',label=f'Test 1D VDZP MAE',marker='x')
plt.plot(VDZP_perc_train,VDZP_Test_RMSE,color='b',label=f'Test 1D VDZP RMSE',marker='+')
plt.plot(VDZP_perc_train,VDZP_train_MAE,color='orange',label=f'Train 1D VDZP MAE',marker='x')
plt.plot(VDZP_perc_train,VDZP_train_RMSE,color='orange',label=f'Train 1D VDZP RMSE',marker='+')


# VDZP_df = pd.read_excel(f'../../VDZP/1D/big_scaled_super_4_3_VDZP_Learning_Curve_1D.xlsx',engine='openpyxl')
# VDZP_test=pd.read_excel('../../VDZP/1D/big_scaled_super_4_3_VDZP_Test_Energies.xlsx',engine='openpyxl').set_index('testsubs').drop(columns='Unnamed: 0')
# VDZP_perc_train=[float(f'{i*100:0.0f}') for i in list(VDZP_df['Num_Train']/VDZP_df['Num_Train'].iloc[-1])]
# VDZP_test_R2=VDZP_df['Test_R2']
# VDZP_Train_R2=VDZP_df['Train_R2']
# VDZP_Test_MAE=VDZP_df['Test_MAE (kcal/mol)']
# VDZP_Test_RMSE=VDZP_df['Test_RMSE (kcal/mol)']
# VDZP_train_MAE=VDZP_df['Train_kcalMAE']
# VDZP_train_RMSE=VDZP_df['Train_kcalRMSE']
# plt.plot(VDZP_perc_train,VDZP_Test_MAE,color='r',label=f'Test 1D VDZP MAE',marker='1',ls='-.')
# plt.plot(VDZP_perc_train,VDZP_Test_RMSE,color='r',label=f'Test 1D VDZP RMSE',marker='2',ls='-.')
# plt.plot(VDZP_perc_train,VDZP_train_MAE,color='g',label=f'Train 1D VDZP MAE',marker='1',ls='-.')
# plt.plot(VDZP_perc_train,VDZP_train_RMSE,color='g',label=f'Train 1D VDZP RMSE',marker='2',ls='-.')

# plt.text(40, 4, f'Average Test Energy: \n {np.average(test.values*627.5):.2f} kcal/mol',fontsize=12,bbox=dict(facecolor='white', alpha=1.0))
plt.title(r'O$_{3}$ 1D (4,3) VDZP Learning Curve',fontsize=14)
plt.xlabel(r'Training Molecules (%)',fontsize=14)
plt.ylabel(r'Error (kcal/mol)',fontsize=14)
plt.xlim(0,103)
plt.ylim(0,3)
plt.fill_between(np.arange(-1,105),0, 1,color='gray', alpha=0.3)
plt.fill_between(np.arange(-1,105),1, 2.3,color='darkgray', alpha=0.3)

plt.legend(loc=1)
plt.tight_layout()
plt.savefig(f'scaled_new_4_3_1D_VDZP_LC.png',dpi=300)
plt.savefig(f'low_res_scaled_new_4_3_1D_VDZP_LC.png')
plt.show()





In [169]:
plt.plot(VDZP_perc_train,VDZP_test_R2,label='test')
plt.plot(VDZP_perc_train,VDZP_Train_R2,label='train')
plt.title(r'O$_{3}$ 1D (4,3) VDZP R$^{2}$ Learning Curve',fontsize=14)
plt.ylabel(r'R$^{2}$',fontsize=14)
plt.xlabel(r'Training Molecules (%)',fontsize=14)
plt.ylim(0,1.1)
plt.xlim(0,101)
plt.legend(loc=2)
plt.tight_layout()
plt.savefig(f'scaled_new_4_3_1D_VDZP_R2.png',dpi=300)
plt.savefig(f'low_res_scaled_new_4_3_1D_VDZP_R2.png')
plt.show()